In [ ]:
# 使用有隐含层的神经网络训练MNIST数据集

In [2]:
# 加载数据集
from tensorflow.examples.tutorials.mnist import input_data
import tensorflow as tf
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)
sess = tf.InteractiveSession()

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
#　设置和初始化网络参数
in_units = 784　　　# 输入节点数
h1_units = 300　　　# 隐含层输出节点数
# 因为激活函数使用ReLU，需要通过正态分布增加一些噪声，打破完全对称，避免0梯度
W1 = tf.Variable(tf.truncated_normal([in_units, h1_units], stddev=0.1))　　# 隐含层权重，初始化为截断正态分布，标准差为0.1
b1 = tf.Variable(tf.zeros([h1_units]))　　　 #　隐含层偏置，初始化为0
# 输出层权重和偏置
W2 = tf.Variable(tf.zeros([h1_units, 10]))
b2 = tf.Variable(tf.zeros([10]))

In [4]:
# 使用Dropout方法，将网络该层的输出节点数据随机丢弃一部分(keep_porb：保留概率)
# 通过增加样本量、减少特征量，创造新随机样本，防止过拟合
# keep_prob通常训练时<1，预测时=1
x = tf.placeholder(tf.float32, [None, in_units])
keep_prob = tf.placeholder(tf.float32)

In [5]:
# 定义模型结构
hidden1 = tf.nn.relu(tf.matmul(x, W1) + b1)   # 隐含层，relu激活函数
hidden1_drop = tf.nn.dropout(hidden1, keep_prob)  # Dropout层
y = tf.nn.softmax(tf.matmul(hidden1_drop, W2) + b2)  # 输出层，softmax激活（分类）

In [7]:
# 定义loss和优化算法
# loss: cross entropy
# 优化器：Adagrad
y_ = tf.placeholder(tf.float32, [None, 10])
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))
train_step = tf.train.AdagradOptimizer(0.3).minimize(cross_entropy)  # 学习速率0.3

In [8]:
# 开始训练，每个batch包含100条样本，训练3000次，相当于一共30w样本，对全数据进行了5epoch迭代
tf.global_variables_initializer().run()
for i in range(3000):
    batch_xs, batch_ys = mnist.train.next_batch(100)
    train_step.run({x:batch_xs, y_:batch_ys, keep_prob:0.75})  # 输入时采用Dropout，保留75%节点，其余为0

In [9]:
# 测试集预测结果
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print(accuracy.eval({x:mnist.test.images, y_:mnist.test.labels, keep_prob:1.0}))

0.9777
